In [1]:
from gurobipy import *
import gurobipy as gp
import pandas as pd

In [2]:
# importing dataset
nba_data = pd.read_csv('OIE_Project_Final_Data.csv')
nba_data.head()

,Unique Identifier,Player,Position,Age,WS,Salary,GS_Percent,G,All-Star
0,jokicni01,Nikola Jokić,C,27,14.9,47607350,1.0,69,1
1,sabondo01,Domantas Sabonis,C,26,12.6,30600000,1.0,79,1
2,butleji01,Jimmy Butler,PF,33,12.3,45183960,1.0,64,1
3,embiijo01,Joel Embiid,C,28,12.3,47607350,1.0,66,1
4,gilgesh01,Shai Gilgeous-Alexander,PG,24,11.4,33386850,1.0,68,1


In [3]:
nba_data['Salary'] = nba_data['Salary'].astype('int64')

In [4]:
# objective function coefficients
max_ws_fun = nba_data['WS']

In [5]:
# creating model
model  = Model()

Restricted license - for non-production use only - expires 2024-10-28


In [6]:
# length of decision variables
num_vars = len(max_ws_fun)
num_vars

391

In [7]:
# adding decision variables to model
vars = model.addVars(num_vars, vtype=gurobipy.GRB.BINARY, name='x')

In [8]:
# setting objective function for maximizing the number of win shares
model.setObjective(gurobipy.quicksum(max_ws_fun[i] * vars[i] 
                                     for i in range(num_vars)), sense=gurobipy.GRB.MAXIMIZE)

In [9]:
# setting the max number of players on the team
max_players = 10
model.addConstr(gurobipy.quicksum(vars[i] for i in range(num_vars)) == max_players)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
# adding constraint to ensure team's salary is not over salary cap
player_salary = nba_data['Salary']
salary_cap = 136021000
team_salary = gurobipy.quicksum(player_salary[i] * vars[i] for i in range(num_vars))
model.addConstr(team_salary <= salary_cap)

<gurobi.Constr *Awaiting Model Update*>

In [11]:
# adding constraint to have 2 players with C position
c_position_constraint = 'C'
max_number_c = 2
count_for_c = gurobipy.quicksum(vars[i] for i in range(num_vars) if nba_data['Position'][i] == c_position_constraint)
model.addConstr(count_for_c <= max_number_c)

<gurobi.Constr *Awaiting Model Update*>

In [12]:
# adding constraint to have 2 players with PF position
pf_position_constraint = 'PF'
max_number_pf = 2
count_for_pf = gurobipy.quicksum(vars[i] for i in range(num_vars) if nba_data['Position'][i] == pf_position_constraint)
model.addConstr(count_for_pf <= max_number_pf)

<gurobi.Constr *Awaiting Model Update*>

In [13]:
# adding constraint to have 2 players with SF position
sf_position_constraint = 'SF'
max_number_sf = 2
count_for_sf = gurobipy.quicksum(vars[i] for i in range(num_vars) if nba_data['Position'][i] == sf_position_constraint)
model.addConstr(count_for_sf <= max_number_sf)

<gurobi.Constr *Awaiting Model Update*>

In [14]:
# adding constraint to have 2 players with SG position
sg_position_constraint = 'SG'
max_number_sg= 2
count_for_sg = gurobipy.quicksum(vars[i] for i in range(num_vars) if nba_data['Position'][i] == sg_position_constraint)
model.addConstr(count_for_sg <= max_number_sg)

<gurobi.Constr *Awaiting Model Update*>

In [15]:
# adding constraint to have 2 players with PG position
pg_position_constraint = 'PG'
max_number_pg = 2
count_for_pg = gurobipy.quicksum(vars[i] for i in range(num_vars) if nba_data['Position'][i] == pg_position_constraint)
model.addConstr(count_for_pg <= max_number_pg)

<gurobi.Constr *Awaiting Model Update*>

In [16]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 391 columns and 1170 nonzeros
Model fingerprint: 0x1e33a115
Variable types: 0 continuous, 391 integer (391 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+07]
  Objective range  [1e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+08]
Found heuristic solution: objective 25.3000000
Presolve removed 0 rows and 34 columns
Presolve time: 0.00s
Presolved: 7 rows, 357 columns, 1068 nonzeros
Found heuristic solution: objective 84.8000000
Variable types: 0 continuous, 357 integer (355 binary)

Root relaxation: objective 8.581776e+01, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

In [17]:
if model.status == gp.GRB.OPTIMAL:
    print("Optimal Solution:")
    for i in range(num_vars):
        if vars[i].x == 1:
            column_entry = nba_data.iloc[i]  # Get the associated row from the DataFrame
            print(f"x[{i}] = 1 (Associated Column Entry: {column_entry})")
    
    print(f'Objective Value = {model.objVal}')
else:
    print("No solution found")

Optimal Solution:
x[0] = 1 (Associated Column Entry: Unique Identifier       jokicni01
Player               Nikola Jokić
Position                        C
Age                            27
WS                           14.9
Salary                   47607350
GS_Percent                    1.0
G                              69
All-Star                        1
Name: 0, dtype: object)
x[4] = 1 (Associated Column Entry: Unique Identifier                  gilgesh01
Player               Shai Gilgeous-Alexander
Position                                  PG
Age                                       24
WS                                      11.4
Salary                              33386850
GS_Percent                               1.0
G                                         68
All-Star                                   1
Name: 4, dtype: object)
x[13] = 1 (Associated Column Entry: Unique Identifier      mobleev01
Player               Evan Mobley
Position                      PF
Age               